In [56]:
from pyntcloud import PyntCloud
from tqdm import tqdm
import open3d as o3d
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

In [57]:
frames_dir = '/home/ty/Downloads/AAI/Lane_Marking_Execution/hamburg/preprocessed/frames'
tf_poses = '/home/ty/Downloads/AAI/Lane_Marking_Execution/hamburg/trajectory_transformed/tf_poses.npy'


In [58]:
def load_states(file):
    data = np.load(file)
    translations = []
    for tf in data:
        translation = tf[:3, 3]
        translations.append(translation)
        
    return data, np.array(translations)

In [59]:
poses, translations = load_states(tf_poses)

In [60]:
print(len(poses))
print(translations.shape)

190
(190, 3)


In [76]:
def read_point_cloud(file_or_cloud):
    if isinstance(file_or_cloud, PyntCloud):
        return file_or_cloud
    elif isinstance(file_or_cloud, str):
        cloud = PyntCloud.from_file(file_or_cloud)
        return cloud
    else:
        raise ValueError("Input must be a file path (str) or a Point Cloud object.")

In [77]:
def pyntcloud_to_open3d(file_or_cloud):
    cloud = read_point_cloud(file_or_cloud)
    cloud_pts = cloud.points[['x', 'y', 'z', 'intensity']].values
    
    xyz = cloud_pts[:, :3]
    open3d_cloud = o3d.geometry.PointCloud()
    open3d_cloud.points = o3d.utility.Vector3dVector(xyz)
    return open3d_cloud

In [85]:
# def visualize_multiple_pcds(*pcds):
#     clouds = [pcd for pcd in pcds]
#     o3d.visualization.draw_geometries(clouds)

In [95]:
# def color_pcd(pcd, clr='r'):
#     if clr == 'r':
#         color = [1, 0, 0]
#         pcd.paint_uniform_color(color)
#     elif clr == 'g':
#         color = [0, 1, 0]
#         pcd.paint_uniform_color(color)
#     elif clr == 'b':
#         color = [0, 0, 1]
#         pcd.paint_uniform_color(color)

#     return pcd

In [98]:
def apply_radial_filter(obj, radius):
    cloud = read_point_cloud(obj)
    pcd = pyntcloud_to_open3d(obj)
    cloud_df = cloud.points
    distances = np.linalg.norm(np.asarray(pcd.points), axis=1)
    mask = distances <= radius
    
    inlier_points_df = cloud_df[mask]
    outlier_points_df = cloud_df[~mask]
    
    # Convert filtered DataFrames back to PyntClouds
    inlier_cloud = PyntCloud(inlier_points_df)
    outlier_cloud = PyntCloud(outlier_points_df)
    
    return inlier_cloud, outlier_cloud
    

In [91]:
def plane_segmentation_mask(obj, dist_thresh, ransac_n, num_iters):
    cloud = read_point_cloud(obj)
    pcd = pyntcloud_to_open3d(obj)
    cloud_df = cloud.points
    
    # Apply plane segmentation
    plane_model, inliers = pcd.segment_plane(distance_threshold=dist_thresh,
                                             ransac_n=ransac_n,
                                             num_iterations=num_iters)
    
    # Create a mask for inliers
    inlier_mask = np.zeros(len(pcd.points), dtype=bool)
    inlier_mask[inliers] = True
    
    # Filtering inlier points based on the mask
    inlier_points_df = cloud_df[inlier_mask]
    outlier_points_df = cloud_df[~inlier_mask]
    
    # Convert filtered DataFrames back to PyntClouds
    inlier_cloud = PyntCloud(inlier_points_df)
    outlier_cloud = PyntCloud(outlier_points_df)
    
    return inlier_cloud, outlier_cloud
    

In [99]:
test_ply_file = '/home/ty/Downloads/AAI/Lane_Marking_Execution/hamburg/data/frames/1693307422024154166.ply'


In [102]:
inlier_cloud, outlier_cloud = apply_radial_filter(test_ply_file, 25)
inlier_pcd = pyntcloud_to_open3d(inlier_cloud)
outlier_pcd = pyntcloud_to_open3d(outlier_cloud)
# print(type(inlier_pcd))
# print(type(outlier_pcd))

outlier_pcd = color_pcd(outlier_pcd, 'r')

In [103]:
visualize_multiple_pcds(inlier_pcd, outlier_pcd)

In [104]:
inlier_cloud, outlier_cloud = plane_segmentation_mask(inlier_cloud, 
                                                     0.1, ransac_n=10, num_iters=1000)

inlier_pcd = pyntcloud_to_open3d(inlier_cloud)
outlier_pcd = pyntcloud_to_open3d(outlier_cloud)
# print(type(inlier_pcd))
# print(type(outlier_pcd))

outlier_pcd = color_pcd(outlier_pcd, 'r')



In [105]:
visualize_multiple_pcds(inlier_pcd, outlier_pcd)

In [61]:
def transform_frames(transformation_matrix, lidar_points):
    # Convert lidar points to homogeneous coordinates
    ones = np.ones((lidar_points.shape[0], 1))
    homogeneous_points = np.hstack((lidar_points[:, :3], ones))

    # Apply the transformation matrix
    transformed_points = transformation_matrix @ homogeneous_points.T

    # Convert back to Cartesian coordinates
    transformed_points = transformed_points.T[:, :3]

    # Append intensity back to the transformed points
    if lidar_points.shape[1] == 4:
        transformed_points = np.hstack((transformed_points, lidar_points[:, 3].reshape(-1, 1)))

    return transformed_points

In [62]:
def generate_map(tf_poses, lidar_frames_dir):
    frames = sorted(os.path.join(lidar_frames_dir, file_name) for file_name in os.listdir(lidar_frames_dir) if
                        file_name.endswith('.ply'))
    
    assert len(tf_poses) == len(
            frames), f"Mismatch between length of poses {len(tf_poses)} and length of lidar frames {len(frames)}"
    
    tf_frame = []
#     tf_ground = []
    for idx, file in tqdm(enumerate(frames), desc='Generating Maps: ', total=len(frames)):
        file_path = frames[idx]
        cloud = PyntCloud.from_file(file_path)
        org_points = cloud.points.values
        tf_pose = tf_poses[idx]
        tf_points = transform_frames(tf_pose, org_points)
        tf_frame.append(tf_points)
    tf_map = np.vstack(tf_frame)
    
    return tf_map, tf_ground
    

In [ ]:
def generate_ground_map(tf_poses, lidar_frames_dir, dist_thresh, ransac_n, num_iters):
    frames = sorted(os.path.join(lidar_frames_dir, file_name) for file_name in os.listdir(lidar_frames_dir) if
                        file_name.endswith('.ply'))
    
    assert len(tf_poses) == len(
            frames), f"Mismatch between length of poses {len(tf_poses)} and length of lidar frames {len(frames)}"
    
    tf_ground = []
    for idx, file in tqdm(enumerate(frames), desc='Generating Ground Map: ', total=len(frames)):
        file_path = frames[idx]
        cloud = PyntCloud.from_file(file_path)
        ground_cloud, _ = plane_segmentation_mask(cloud, dist_thresh, ransac_n, num_iters)
        ground_points = ground_cloud.points.values
        tf_pose = tf_poses[idx]
        tf_ground_points = transform_frames(tf_pose, ground_points)
        tf_ground.append(tf_ground_points)
        
    tf_ground = np.vstack(tf_ground)
    
    return tf_ground
    

In [63]:
tf_map = generate_map(poses, frames_dir)

Generating Map: 100%|█████████████████████████| 190/190 [00:04<00:00, 42.35it/s]


In [73]:
def write_maps(tf_map, tf_ground_map, map_file_path, colored_map=True):
    map_cloud = PyntCloud(pd.DataFrame(tf_map, columns=['x', 'y', 'z', 'intensity']))
    map_cloud.to_file(map_file_path)
    print(f"Map file saved to {map_file_path}")
    ground_cloud = PyntCloud(pd.DataFrame(tf_ground_map, columns=['x', 'y', 'z', 'intensity']))
    ground_cloud.to_file(ground_map_file_path)
    print(f"Ground Map file saved to {ground_map_file_path}")
    if colored_map:
        z_values = tf_map[:, 2]
        
        norm = Normalize(vmin=np.min(z_values), 
                         vmax=np.max(z_values))
        
        colors = plt.cm.nipy_spectral(norm(z_values))[:, :3]
#         print(colors.shape)
        # Combine coordinates and colors
        data = np.hstack((tf_map[:, :3], colors))
        columns = ["x", "y", "z", "red", "green", "blue"]
        
        # Create a DataFrame and a new PyntCloud for the colored map
        colored_cloud = PyntCloud(pd.DataFrame(data, columns=columns))
        
        # Construct the file path for the colored map
        file_name = os.path.basename(os.path.splitext(map_file_path)[0]) + '_colored.ply'
        dir_path = os.path.dirname(map_file_path)
        file_path = os.path.join(dir_path, file_name)
        
        # Save the colored point cloud to a .ply file
        colored_cloud.to_file(file_path)
        print(f"Colored Map file saved to {file_path}")
        

In [74]:
map_file_path = '/home/ty/Downloads/AAI/test/map.ply'

In [75]:
write_maps(tf_map, map_file_path, True)

Map file saved to /home/ty/Downloads/AAI/test/map.ply
Colored Map file saved to /home/ty/Downloads/AAI/test/map_colored.ply
